# Restaurant Sales Analysis — Machine Learning Pipeline

**Dataset:** `restaurant_sales_data.csv`  
**Goal:** Build a complete end-to-end Machine Learning pipeline to predict restaurant sales using real-world best practices.

---

## Notebook Overview

This notebook implements a full ML workflow:

1. **Data Loading & Exploration**
2. **Data Cleaning**
3. **Feature Engineering**
4. **Train–Test Split (strictly before preprocessing)**
5. **Preprocessing Pipelines (imputation, encoding, scaling)**
6. **Model Training**
7. **Model Evaluation & Comparison**
8. **Saving Models & Preprocessors**

This workflow follows **industry-standard practices**, including:
- avoiding data leakage  
- reproducible data pipelines  
- separating transformations for linear vs. tree-based models  


-
-

---
## 1. Importing Libraries
Below we import all required Python libraries for data analysis, visualization, preprocessing, and machine learning.


In [1]:
# Basic utilities
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer, SimpleImputer

# Models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")


-
-

---
## 2. Loading the Dataset

The dataset is stored in the GitHub repository under the `data/` folder.  
Using GitHub raw links ensures that the notebook remains **fully reproducible** anywhere.


In [2]:
# Loading dataset from github
url = "https://raw.githubusercontent.com/aymanberri/restaurant-sales-ml/main/data/restaurant_sales_data.csv"

df = pd.read_csv(url)

# Display basic info
df.head()


,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,ORD_705844,CUST_092,Side Dishes,Side Salad,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,NaN,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


---
## 3. Initial Dataset Overview

Here we inspect:
- the shape of the dataset  
- data types  
- missing values  
- basic statistical summary  


In [3]:
print("Shape:", df.shape)
print("\nInfo:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())
df.describe(include="all")


Shape: (17534, 9)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order ID        17534 non-null  object 
 1   Customer ID     17534 non-null  object 
 2   Category        17534 non-null  object 
 3   Item            15776 non-null  object 
 4   Price           16658 non-null  float64
 5   Quantity        17104 non-null  float64
 6   Order Total     17104 non-null  float64
 7   Order Date      17534 non-null  object 
 8   Payment Method  16452 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.2+ MB
None

Missing Values:
Order ID             0
Customer ID          0
Category             0
Item              1758
Price              876
Quantity           430
Order Total        430
Order Date           0
Payment Method    1082
dtype: int64


,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
count,17534,17534,17534,15776,16658.000000,17104.000000,17104.000000,17534,16452
unique,17534,100,5,26,NaN,NaN,NaN,730,3
top,ORD_680707,CUST_066,Main Dishes,Pasta Alfredo,NaN,NaN,NaN,2023-11-25,Credit Card
freq,1,207,3551,998,NaN,NaN,NaN,42,5504
mean,NaN,NaN,NaN,NaN,6.586325,3.014149,19.914494,NaN,NaN
std,NaN,NaN,NaN,NaN,4.834652,1.414598,18.732549,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,3.000000,2.000000,7.500000,NaN,NaN
50%,NaN,NaN,NaN,NaN,5.000000,3.000000,15.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,7.000000,4.000000,25.000000,NaN,NaN


---
## 4. Data Cleaning

In this section, we:
- Remove duplicate rows  
- Standardize column names  
- Convert `Order Date` to datetime  
- Check for invalid or placeholder values  
- Do *light* cleaning only — full preprocessing will be done later  


### 1. Standardize column names

In [4]:
# Make a copy to avoid modifying original df accidentally
df_clean = df.copy()


# Standardising the column names
df_clean.columns = (
    df_clean.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df_clean.head()


,order_id,customer_id,category,item,price,quantity,order_total,order_date,payment_method
0,ORD_705844,CUST_092,Side Dishes,Side Salad,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,NaN,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


Using lowercase_with_underscores makes the dataset easier to work with and avoids errors during preprocessing pipelines.


### 2. Remove duplicates

In [5]:
initial_rows = df_clean.shape[0]

df_clean = df_clean.drop_duplicates()

print("Rows before:", initial_rows)
print("Rows after :", df_clean.shape[0])
print("Duplicates removed:", initial_rows - df_clean.shape[0])


Rows before: 17534
Rows after : 17534
Duplicates removed: 0


### 3. Convert `order_date` to datetime

_This enables extracting date features later (month, day, weekday, etc.)_

In [6]:
df_clean['order_date'] = pd.to_datetime(df_clean['order_date'], errors='coerce')

df_clean['order_date'].head()


,order_date
0,2023-12-21
1,2023-05-19
2,2023-09-27
3,2022-08-09
4,2022-05-15


### 4. Quick check for placeholder values

_check for basic data quality issues, just to know if they exist or not_

In [7]:
# Check for weird price values
print("Negative or zero PRICES:", (df_clean['price'] <= 0).sum())

# Check quantity issues
print("Negative or zero QUANTITIES:", (df_clean['quantity'] <= 0).sum())

# Check totals
print("Negative or zero TOTALS:", (df_clean['order_total'] <= 0).sum())


Negative or zero PRICES: 0
Negative or zero QUANTITIES: 0
Negative or zero TOTALS: 0


_non exist 👍_

### 5. View missing values (of the cleaned dataset)

We already saw missing values earlier, but now lets inspect them after cleaning:

In [8]:
df_clean.isnull().sum()


,0
order_id,0
customer_id,0
category,0
item,1758
price,876
quantity,430
order_total,430
order_date,0
payment_method,1082


### 6. Save Cleaned Dataset

In [9]:
df_clean.to_csv("cleaned_restaurant_sales.csv", index=False)
print("Saved cleaned dataset as cleaned_restaurant_sales.csv")


Saved cleaned dataset as cleaned_restaurant_sales.csv


In [10]:
df_clean.isnull().sum()


,0
order_id,0
customer_id,0
category,0
item,1758
price,876
quantity,430
order_total,430
order_date,0
payment_method,1082


---
## 5. Feature Engineering & Preprocessing

transform the cleaned dataset to a format suitable for machine learning.

Steps:
1. Decide target variable
2. Create new features from existing columns
3. Split into training and testing sets
4. Apply preprocessing pipelines:
   - Imputation for missing values
   - Encoding categorical features
   - Scaling numerical features
5. Save processed data and transformers for modeling

### 1. Target Variable    

*We will predict `order_total` (continuous value) using features like category, item, price, quantity, payment method, and date-related features.*
     

Looking at the dataset:

- order_total = price * quantity (or given directly)

- This is numeric, so its a regression problem


In [11]:
TARGET = "order_total"


### Step2. Feature Engineering

extract date components from o`rder_date`:

In [12]:
# Make a copy for feature engineering, keep things seperate and clean
df_fe = df_clean.copy()

# Date features
df_fe['order_year'] = df_fe['order_date'].dt.year
df_fe['order_month'] = df_fe['order_date'].dt.month
df_fe['order_day'] = df_fe['order_date'].dt.day
df_fe['order_weekday'] = df_fe['order_date'].dt.weekday  # 0=Monday, ...

# Drop order_date itself, we dont need it anymore
df_fe = df_fe.drop(columns=['order_date'])


new features created:
- `order_year`, `order_month`, `order_day`, `order_weekday`  

### 3. Train/test splitting

Before splitting, we must remove any rows where `order_total` is missing, ensuring the target data has no NaN values. Because models can't train on NaN values.

In [13]:
# Drop rows where target is missing
df_fe = df_fe.dropna(subset=[TARGET])

spliting before any preprocessing to avoid data leakage. ALWAYS!!    
and fit transformers with only the training data    
Apply the transformations later on the test data.

In [14]:
X = df_fe.drop(columns=[TARGET, 'order_id', 'customer_id', 'item'])  # remove IDs / raw item name
y = df_fe[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training set shape:", X_train.shape)
print("Test set shape    :", X_test.shape)


Training set shape: (13683, 8)
Test set shape    : (3421, 8)


_during the splitting, I:_    
IDs:  dropped them, theyre meaningless for my case     
High-cardinality categorical (item): dropped bec it is a categorical feature with high cardinality (many unique item names)

### 4.Separate Numerical & Categorical Features

separation must happen bec ML algorithms treat them differently. ex: algorithms can't use categorical feature directly, we must encode. And for the numerical we might have to scale

In [15]:
# identify and sep. numerical and categorical columns
num_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

print("Numerical columns:", num_cols)
print("Categorical columns:", cat_cols)


Numerical columns: ['price', 'quantity']
Categorical columns: ['category', 'payment_method']


### 5. Imputation, Encoding, Scaling (+ Saving)

Imputation (fill gaps):

- Numerical: `KNNImputer` (prediction using nearest neighbors, best for my case)
- Categorical: Mode (most frequent, just fills missing values with the most frequent)

Encoding:
- `LabelEncoder` for categorical features (simple and works for tree-based models)

Scaling:
- `StandardScaler` only for linear models (prevent on feature from dominating)

In [16]:
# Numerical imp
num_imputer = KNNImputer()
X_train_num = pd.DataFrame(num_imputer.fit_transform(X_train[num_cols]), columns=num_cols)

# Categorical imp
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_cat = pd.DataFrame(cat_imputer.fit_transform(X_train[cat_cols]), columns=cat_cols)

# Encoding categorical features
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X_train_cat[col] = le.fit_transform(X_train_cat[col])
    encoders[col] = le

# Combine numerical + categorical
X_train_processed = pd.concat([X_train_num, X_train_cat], axis=1)


After the tranformers learn and transform the training data, we **ONLY** transform the test data (without learning, only transform() on the test data to prevent data leakage)

In [17]:
# Numerical
X_test_num = pd.DataFrame(num_imputer.transform(X_test[num_cols]), columns=num_cols)

# Categorical
X_test_cat = pd.DataFrame(cat_imputer.transform(X_test[cat_cols]), columns=cat_cols)
for col in cat_cols:
    le = encoders[col]
    X_test_cat[col] = le.transform(X_test_cat[col])

# Combine
X_test_processed = pd.concat([X_test_num, X_test_cat], axis=1)


Finally we scale the numerical values so they are on a similar range. (preventing certain features from dominating, bec many algorithms are sensitive to feature sclaes)    

This scaling is mainly for Linear models (linear reg, logistic reg, SVMs, KNN)

In [18]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_processed), columns=X_train_processed.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_processed), columns=X_test_processed.columns)


And save the stuff and transformers...    
"Saving transformers ensures that future data is treated exactly like the training data, which is critical for model correctness and reproducibility."

In [19]:
import pickle #allows saving py objects to a file and load them later

# processed datasets
X_train_scaled.to_csv("X_train_scaled.csv", index=False)   #scaled features for linear models
X_test_scaled.to_csv("X_test_scaled.csv", index=False)     #scaled features for linear models

X_train_processed.to_csv("X_train_tree.csv", index=False)   # Encoded, imputed features for tree-based models like DTs and Random Forests (no scaling needed).
X_test_processed.to_csv("X_test_tree.csv", index=False)     #Encoded, imputed features for tree-based models (no scaling needed).

y_train.to_csv("y_train.csv", index=False)  #target for training
y_test.to_csv("y_test.csv", index=False)    #target for testing

# transformers
with open("num_imputer.pkl", "wb") as f:
    pickle.dump(num_imputer, f)
with open("cat_imputer.pkl", "wb") as f:
    pickle.dump(cat_imputer, f)
with open("encoders.pkl", "wb") as f:
    pickle.dump(encoders, f)
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Saved preprocessed datasets and transformers.")


Saved preprocessed datasets and transformers.


---
## 6. Model Training & Evaluation

Now we train three regression models to predict `order_total`:

1. **Linear Regression** (requires scaled data)  
2. **Decision Tree Regressor** (tree-based, does not require scaling)  
3. **Random Forest Regressor** (more robust than DT)

We will evaluate them using:
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- Mean Absolute Error (MAE)
- R² Score

Finally, we compare performances and save the best model.

### 1. Prepare Data for Models

In [22]:
# Linear Regression (scaled data required)

X_train_lr = X_train_scaled  # Training features
X_test_lr  = X_test_scaled   # Test features

# Tree-based models (no scaling required, bec they can handle raw numeric + encoded categorical data)
X_train_tree = X_train_processed  # Training features for tree-based models
X_test_tree  = X_test_processed   # Test features for tree-based models

### 2. Train Models

In [25]:
# Linear Regression
lr_model = LinearRegression() #create obj
lr_model.fit(X_train_lr, y_train) # training
y_pred_lr = lr_model.predict(X_test_lr) # prediction

# Decision Tree
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train_tree, y_train)
y_pred_dt = dt_model.predict(X_test_tree)

# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_tree, y_train)
y_pred_rf = rf_model.predict(X_test_tree)


### 3. Evaluate Models


In [29]:
# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)  # manually compute RMSE
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2}

# Run the evaluation function on the three models
results = {
    "Linear Regression": evaluate_model(y_test, y_pred_lr),
    "Decision Tree"   : evaluate_model(y_test, y_pred_dt),
    "Random Forest"   : evaluate_model(y_test, y_pred_rf)
}

# Convert to DataFrame for nice display
results_df = pd.DataFrame(results).T
results_df


,MSE,RMSE,MAE,R2
Linear Regression,53.967422,7.346252,4.687174,0.845971
Decision Tree,0.905842,0.951758,0.109907,0.997415
Random Forest,0.867462,0.931376,0.108612,0.997524


_for MSE, RMSE, and MAE lower = BETTER_


| Metric | Explanation                                               |
| ------ | ------------------------------------------------------------- |
| MSE    | Square your mistakes and average them, (big mistakes greatly affect) |
| RMSE   | Average mistake in dollars, easier to understand (just the squre root of MSE)|
| MAE    | Average mistake in dollars, treats all mistakes equally (absolute value of the errors)|
| R²     | How smart your model is compared to guessing the average|


_Rule of thumb, the model with lowest RMSE and MAE and highest R2 is the best._
Random Forest is the best performing model:
   - Lowest RMSE and MAE
   - Highest R2 (0.998)

### 4. Save the best model (highest R2)

In [30]:
best_model_name = results_df['R2'].idxmax()
print("Best Model:", best_model_name)

# Save the best model
import pickle

if best_model_name == "Linear Regression":
    with open("best_model.pkl", "wb") as f:
        pickle.dump(lr_model, f)
else:
    with open("best_model.pkl", "wb") as f:
        if best_model_name == "Decision Tree":
            pickle.dump(dt_model, f)
        else:
            pickle.dump(rf_model, f)

print("Best model saved as 'best_model.pkl'")

Best Model: Random Forest
Best model saved as 'best_model.pkl'


---
## End



Voila    
Overall, this project shows how a **structured, end-to-end ML workflow** can turn messy data into actionable insights. Its a solid foundation for tackling real-world data problems, and honestly, its kind of fun to see a model actually learn patterns in restaurant sales
